In [46]:
import random
from typing import List

MAX_BPS = 10_000
INTERNAL_SECONDS_SINCE_DEPLOY = 0
INITIAL_FEED = 1000

name_list = [
    "Finley",
    "Khan",
    "Jaycee",
    "Bowers",
    "Dwayne",
    "Mosley",
    "Abigail",
    "Shaw",
    "Madison",
    "Ritter",
    "Adolfo",
    "Travis",
    "Roderick",
    "Chavez",
    "Riya",
    "Blevins",
    "Tamia",
    "Richmond",
    "Elisha",
    "Huffman",
    "Yurem",
    "Mitchell",
    "Amanda",
    "Frye",
    "Kara",
    "Matthews",
    "Beckham",
    "Mercer",
    "Carolina",
    "Reynolds",
    "Skye",
    "Blackburn",
    "Carlos",
    "Ashley",
    "Jaslyn",
    "Tanner",
    "Theodore",
    "Bruce",
    "Jensen",
    "Edwards",
    "Shane",
    "Morton",
    "Logan",
    "Fowler",
    "Destiney",
    "Patterson",
    "Maximilian",
    "Thornton",
]

class Trove:
    def __init__(self, owner, system: "Ebtc"):
        self.deposits = 0
        self.debt = 0
        self.last_update_ts = INTERNAL_SECONDS_SINCE_DEPLOY
        self.owner = owner
        self.system = system
        self.system.add_trove(self)

    def __repr__(self):
        return str(self.__dict__)

    def local_collateral_ratio(self):
        return self.debt * MAX_BPS / self.deposits

    def deposit(self, amount):
        self.system.total_deposits += amount
        self.deposits += amount
        self.owner.reduce_balance(self, amount)

    def withdraw(self, amount):
        self.deposits -= amount
        assert self.is_solvent()

    def borrow(self, amount):
        self.debt += amount
        assert self.is_solvent()
        self.system.total_debt += amount
        assert self.system.is_solvent()

    def repay(self, amount):
        ## TODO
        return 0

    def liquidate(self, amount, caller):
        ## Only if not owner
        if caller == self.owner:
            return False

        return 0

    ## SECURITY CHECKS
    def is_trove(self):
        return True

    def max_borrow(self):
        return self.deposits * self.system.feed

    def is_solvent(self):
        ## Strictly less to avoid rounding or w/e
        return self.debt < self.max_borrow()

ETH_BTC_PRICE = 14  # Assuming 1BTC is 14ETH

class Ebtc:
    def __init__(self):
        self.MAX_LTV = 15000  ## 150%
        self.FEE_PER_SECOND = 0  ## No fee for borrows
        self.ORIGINATION_FEE = 50  ## 50BPS

        self.total_deposits = 0
        self.total_debt = 0
        self.feed = INITIAL_FEED
        self.troves = []  # type: List["Trove"]

    def __repr__(self):
        return str(self.__dict__)

    def collateral_ratio(self):
        return self.total_debt * MAX_BPS / self.total_deposits

    def redeem(self, amount: int, user: "User"):
        """
        Redeem with recursion until amount is fully redeemed
        """
        min_cr = min([trove.local_collateral_ratio() for trove in self.troves])
        riskiest_trove = [trove for trove in self.troves if trove.local_collateral_ratio() <= min_cr][0]
        redeem_amount = min(amount, riskiest_trove.deposits)
        riskiest_trove.withdraw((redeem_amount * ETH_BTC_PRICE))  # TODO: Learn about the price for BTC properly
        user.increase_balance(self, redeem_amount * ETH_BTC_PRICE)

        left_to_be_redeemed = amount - redeem_amount
        if left_to_be_redeemed:
            self.redeem(left_to_be_redeemed, user)

    def max_borrow(self):
        return self.total_debt * self.feed

    def is_in_emergency_mode(self):
        ## TODO:
        return False

    def is_solvent(self):
        ## NOTE: Strictly less to avoid rounding, etc..
        return self.total_debt < self.max_borrow()

    def set_feed(self, value):
      self.feed = value

    def add_trove(self, trove: Trove) -> None:
        self.troves.append(trove)


class User:
    def __init__(self, initial_balance_collateral):
        self.collateral = initial_balance_collateral
        self.name = random.choice(name_list)

    def __repr__(self):
        return str(self.__dict__)

    def increase_balance(self, caller, amount):
        self.collateral += amount

    def reduce_balance(self, caller, amount):
        self.collateral -= amount

    def get_balance(self):
        return self.collateral


## POOL For Swap
class UniV2Pool:
  def __init__(self, start_x, start_y, start_lp):
    ## NOTE: May or may not want to have a function to hardcode this
    self.reserve_x = start_x
    self.reserve_y = start_y
    self.total_supply = start_lp

  def k(self):
    return self.x * self.y

  def get_price_out(self, is_x, amount):
    if is_x:
      return self.get_price(amount, self.reserve_x, self.reserve_y)
    else:
      return self.get_price(amount, self.reserve_y, self.reserve_x)

  ## UniV2 Formula, can extend the class and change this to create new pools
  def get_price(self, amount_in, reserve_in, reserve_out):
      amountInWithFee = amount_in * 997
      numerator = amountInWithFee * reserve_out
      denominator = reserve_in * 1000 + amountInWithFee
      amountOut = numerator / denominator

      return amountOut

  def withdraw_lp(self):
    ## TODO
    return False

  def lp(self):
    ## TODO
    return False


## Borrows and Holds
class Borrower(User):
    def step(self):
        pass

## Borrow and Sells when price is higher
class LongArbitrager(User):
    def step(self):
        pass

## Buys when cheap and sells when higher
class ShortArbitrager(User):
    def step(self):
        pass

## Does both arbitrages
class Trader(User):
    def step(self):
        pass

# init the system
ebtc = Ebtc()

# init a user with a balance of 100
user_1 = User(100)

# init a trove for this user
trove_1 = Trove(user_1, ebtc)
print(trove_1.__dict__)

# make a deposit into the system
trove_1.deposit(25)
print(trove_1.__dict__)

# borrow against this deposit
trove_1.borrow(12.5)
print(trove_1.__dict__)

## Test for Feed and solvency
assert trove_1.is_solvent()

ebtc.redeem(100, user_1)

{'deposits': 0, 'debt': 0, 'last_update_ts': 0, 'owner': {'collateral': 100, 'name': 'Ashley'}, 'system': {'MAX_LTV': 15000, 'FEE_PER_SECOND': 0, 'ORIGINATION_FEE': 50, 'total_deposits': 0, 'total_debt': 0, 'feed': 1000, 'troves': [{...}]}}
{'deposits': 25, 'debt': 0, 'last_update_ts': 0, 'owner': {'collateral': 75, 'name': 'Ashley'}, 'system': {'MAX_LTV': 15000, 'FEE_PER_SECOND': 0, 'ORIGINATION_FEE': 50, 'total_deposits': 25, 'total_debt': 0, 'feed': 1000, 'troves': [{...}]}}
{'deposits': 25, 'debt': 12.5, 'last_update_ts': 0, 'owner': {'collateral': 75, 'name': 'Ashley'}, 'system': {'MAX_LTV': 15000, 'FEE_PER_SECOND': 0, 'ORIGINATION_FEE': 50, 'total_deposits': 25, 'total_debt': 12.5, 'feed': 1000, 'troves': [{...}]}}


AssertionError: 